## Object Detection


## Owl2
- https://huggingface.co/google/owlv2-base-patch16
- https://huggingface.co/google/owlv2-large-patch14
- https://huggingface.co/google/owlv2-large-patch14-ensemble

#### Results from experiments
- Use larger images and smaller models

In [ ]:
import json

from os import listdir, makedirs, path

from PIL import Image as PImage, ImageOps as PImageOps, ImageDraw as PImageDraw

from models.Owlv2 import Owlv2

from params.detect import OBJS_LABELS_IN as OBJS_LABELS, OBJS_THOLDS
from params.wiki import MUSEUMS

In [ ]:
wiki_museum = MUSEUMS["pinacoteca"]

In [ ]:
WIKI_DATA_DIR = f"./metadata/json/{wiki_museum['dir']}"
WIKI_INFO_PATH = path.join(WIKI_DATA_DIR, f"{wiki_museum['file']}.json")

WIKI_OBJECT_DIR = path.join(WIKI_DATA_DIR, "objects")
makedirs(WIKI_OBJECT_DIR, exist_ok=True)

IMG_DIR = f"../../imgs/{wiki_museum['dir']}"
IMG_DIR_500 = path.join(IMG_DIR, "500")
IMG_DIR_900 = path.join(IMG_DIR, "900")

In [ ]:
wiki_data = {}

if (path.isfile(WIKI_INFO_PATH)):
  with open(WIKI_INFO_PATH, "r") as ifp:
    wiki_data = json.load(ifp)

qids = sorted(list(wiki_data.keys()))

In [ ]:
owl = Owlv2("google/owlv2-base-patch16")

In [ ]:
for cnt,qid in enumerate(qids):
  if cnt % 100 == 0:
    print(cnt)

  input_file_path = path.join(IMG_DIR_900, f"{qid}.jpg")
  output_file_path = path.join(WIKI_OBJECT_DIR, f"{qid}.json")

  if path.isfile(output_file_path):
    continue

  image = PImageOps.exif_transpose(PImage.open(input_file_path).convert("RGB"))

  image_boxes = []
  for labels,tholds in zip(OBJS_LABELS, OBJS_THOLDS):
    obj_boxes = owl.all_objects(image, labels, tholds)
    image_boxes += obj_boxes

  with open(output_file_path, "w", encoding="utf-8") as of:
    json.dump(image_boxes, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

### Visualize

In [ ]:
obj_files = sorted([f for f in listdir(WIKI_OBJECT_DIR) if f.endswith(".json")])

In [ ]:
for fname in obj_files:
  with open(path.join(WIKI_OBJECT_DIR, fname), "r") as inp:
    iboxes = json.load(inp)

  if len(iboxes) < 1:
    continue

  image_file_path = path.join(IMG_DIR_900, fname.replace(".json", ".jpg"))
  image = PImageOps.exif_transpose(PImage.open(image_file_path).convert("RGB"))
  iw,ih = image.size
  draw = PImageDraw.Draw(image)

  for box in iboxes:
    label, (x0,y0,x1,y1) = box["label"], box["box"]
    draw.rectangle(((x0*iw, y0*ih), (x1*iw, y1*ih)), outline=(255, 0, 0), width=2)

  display(image)

### Mosaic

In [ ]:
IMG_DIR = f"../../imgs/{wiki_museum['dir']}"
IMG_DIR_500 = path.join(IMG_DIR, "500")
IMG_DIR_900 = path.join(IMG_DIR, "900")
IMG_DIR_FULL = path.join(IMG_DIR, "full")

IMG_DIR_FLORA = path.join(IMG_DIR, "flora-mosaic")
makedirs(IMG_DIR_FLORA, exist_ok=True)

In [ ]:
obj_files = sorted([f for f in listdir(WIKI_OBJECT_DIR) if f.endswith(".json")])

In [ ]:
MAX_HEIGHT = 100

total_width = 0

for fname in obj_files:
  with open(path.join(WIKI_OBJECT_DIR, fname), "r") as inp:
    iboxes = json.load(inp)

  if len(iboxes) < 1:
    continue

  image_file_path = path.join(IMG_DIR_FULL, fname.replace(".json", ".jpg"))
  image = PImageOps.exif_transpose(PImage.open(image_file_path).convert("RGB"))
  iw,ih = image.size

  for bidx,box in enumerate(iboxes):
    (x0,y0,x1,y1) = box["box"]
    bimg = image.crop((x0*iw, y0*ih, x1*iw, y1*ih))
    biw, bih = bimg.size
    if bih > MAX_HEIGHT:
      bimg = bimg.resize((int(biw * MAX_HEIGHT / bih), MAX_HEIGHT))
    
    bifname = f"{fname.replace('.json', '')}_{('0000'+str(bidx))[-3:]}.jpg"
    bimg.save(path.join(IMG_DIR_FLORA, bifname))
    
    total_width += bimg.size[0]

print(total_width)

### Create Mosaic

In [ ]:
import numpy as np

In [ ]:
MAX_HEIGHT = 100
total_width = 218903
F2x1 = 2.15

total_pxs = total_width * MAX_HEIGHT
print(total_pxs)

mdim_1x1 = round(total_pxs ** 0.5)
mdim_2x1 = round((total_pxs/F2x1)**0.5)

print(mdim_1x1, "->", mdim_1x1*mdim_1x1)
print(mdim_2x1, "->", F2x1*mdim_2x1*mdim_2x1)

In [ ]:
IMG_SIZES = {
  "1x1": (4680, 4680),
  "2x1": (3190, int(F2x1*3190))
}

In [ ]:
flora_files = sorted([f for f in listdir(IMG_DIR_FLORA) if f.startswith("Q") and f.endswith(".jpg")])
print(len(flora_files))

In [ ]:
MIN_WIDTH_F = 0.98

for size_tag,mimg_size in IMG_SIZES.items():
  cx,cy = 0,0
  mimg = PImage.fromarray(np.zeros(mimg_size)).convert("RGB")
  miw, mih = mimg.size

  for fname in flora_files:
    fimg = PImage.open(path.join(IMG_DIR_FLORA, fname))
    fiw, fih = fimg.size

    if (cx+fiw) > miw and (cx > MIN_WIDTH_F*miw):
      cx = 0
      cy += MAX_HEIGHT
    if cy >= mih:
      print("breaking")
      break

    mimg.paste(fimg, (cx,cy))
    cx += fiw

  mimg = mimg.crop((0, 0, int(MIN_WIDTH_F*miw), cy))
  mimg.save(path.join(IMG_DIR_FLORA, f"flora_{size_tag}.jpg"))

## SigLip2

For embedding

Large patch 16x16:
- https://huggingface.co/google/siglip2-large-patch16-384
- https://huggingface.co/google/siglip2-large-patch16-512

Giant 16x16:
- https://huggingface.co/google/siglip2-giant-opt-patch16-256
- https://huggingface.co/google/siglip2-giant-opt-patch16-384


